In [1]:
import os
import getpass
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass('LANGCHAIN_API_KEY:')

from langchain_openai import ChatOpenAI
os.environ["OPENAI_API_KEY"] = getpass.getpass('OPENAI_API_KEY:')
# 配置模型地址
os.environ["OPENAI_API_BASE"] = getpass.getpass('OPENAI_API_BASE:')
# 配置模型版本
model = ChatOpenAI(model="gpt-3.5-turbo")

ValidationError: 1 validation error for ChatOpenAI
__root__
  Did not find openai_api_key, please add an environment variable `OPENAI_API_KEY` which contains it, or pass `openai_api_key` as a named parameter. (type=value_error)

In [ ]:
# 构建提示模板
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
) 
chain = prompt | model
response = chain.invoke({"messages": [HumanMessage(content="hi,i am bob")]})
response.content

In [ ]:
# 构建消息历史
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store = {}

def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]

with_message_history = RunnableWithMessageHistory(model,get_session_history)
config = {'configurable': {"session_id": "abc5"}}
response = with_message_history.invoke([HumanMessage(content="hi,i am jim")],config)
response.content

In [ ]:
# 通过config使用历史消息
response = with_message_history.invoke([HumanMessage(content="what is my name?")],config)
response.content

In [ ]:
# 构建提示模版，带有变量language
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)
chain = prompt | model

In [ ]:
# 按照中文回答问题
chain.invoke({"messages": [HumanMessage(content="hi,i am bob")],"language":"chinese"}).content

In [ ]:
# 构建历史消息
with_message_history = RunnableWithMessageHistory(chain,get_session_history,input_messages_key="messages")
# 创建模版
config = {'configurable': {"session_id": "abc11"}}
# 带有历史消息的回答
with_message_history.invoke(
    {'messages':[HumanMessage(content="hi,i am todd")],"language":"chinese"}
    ,config
)
response = with_message_history.invoke(
    {'messages':[HumanMessage(content="what is my name?")],"language":"chinese"}
    ,config
)
response.content
